# IMPORTS

In [1]:
import ccxt
import pandas as pd
import time
import datetime as dt
from config import myconfig

# INITIALIZE

In [2]:
exchange = ccxt.binance({
    "apiKey": myconfig.API_KEY,
    "secret": myconfig.API_SECRET
})

# GET MARKET FEES

In [3]:
marketFees = exchange.fetch_fees()['trading']

In [4]:
marketFeesDF = pd.DataFrame.from_dict(marketFees, orient= 'index').drop(columns= 'info')

# GET MARKETS

In [5]:
markets = exchange.fetchMarkets()
market_symbols = [market['symbol'] for market in markets]
print(f'No. of market symbols: {len(market_symbols)}')
print(f'Sample:{market_symbols[0:5]}')

No. of market symbols: 2027
Sample:['ETH/BTC', 'LTC/BTC', 'BNB/BTC', 'NEO/BTC', 'QTUM/ETH']


In [6]:
markets_df = pd.DataFrame.from_dict(markets)
list(markets_df)

['id',
 'lowercaseId',
 'symbol',
 'base',
 'quote',
 'settle',
 'baseId',
 'quoteId',
 'settleId',
 'type',
 'spot',
 'margin',
 'swap',
 'future',
 'delivery',
 'option',
 'active',
 'contract',
 'linear',
 'inverse',
 'taker',
 'maker',
 'contractSize',
 'expiry',
 'expiryDatetime',
 'strike',
 'optionType',
 'precision',
 'limits',
 'info']

In [7]:
spot_market = markets_df[markets_df.type == 'spot']
spot_market.drop(columns=[
    'delivery', 'option', 'active', 'contract', 'linear',
    'inverse', 'taker', 'maker', 'contractSize', 'expiry',
    'expiryDatetime', 'margin', 'swap', 'future', 'strike',
    'optionType', 'precision', 'limits', 'info'], inplace= True)
spot_market.head()

,id,lowercaseId,symbol,base,quote,settle,baseId,quoteId,settleId,type,spot
0,ETHBTC,ethbtc,ETH/BTC,ETH,BTC,None,ETH,BTC,None,spot,True
1,LTCBTC,ltcbtc,LTC/BTC,LTC,BTC,None,LTC,BTC,None,spot,True
2,BNBBTC,bnbbtc,BNB/BTC,BNB,BTC,None,BNB,BTC,None,spot,True
3,NEOBTC,neobtc,NEO/BTC,NEO,BTC,None,NEO,BTC,None,spot,True
4,QTUMETH,qtumeth,QTUM/ETH,QTUM,ETH,None,QTUM,ETH,None,spot,True


In [8]:
spot_symbols = list(spot_market.symbol)
len(spot_symbols)

2027

# STEP 1: GET ALL THE CRYPTO COMBINATIONS FOR USDT

In [9]:
def get_crypto_combinations(market_symbols, base):
    combinations = []
    for sym1 in market_symbols:
        
        sym1_token1 = sym1.split('/')[0]
        sym1_token2 = sym1.split('/')[1]
        
        if (sym1_token2 == base):
            for sym2 in market_symbols:
                sym2_token1 = sym2.split('/')[0]
                sym2_token2 = sym2.split('/')[1]
                if (sym1_token1 == sym2_token2):
                    for sym3 in market_symbols:
                        sym3_token1 = sym3.split('/')[0]
                        sym3_token2 = sym3.split('/')[1]
                        if((sym2_token1 == sym3_token1) and (sym3_token2 == sym1_token2)):
                            combination = {
                                'base':sym1_token2,
                                'intermediate':sym1_token1,
                                'ticker':sym2_token1,
                            }
                            combinations.append(combination)
                

    return combinations
        
wx_combinations_usdt = get_crypto_combinations(spot_symbols,'USDT')


In [10]:
print(f'No. of crypto combinations: {len(wx_combinations_usdt)}')

cominations_df = pd.DataFrame(wx_combinations_usdt)
cominations_df.head()

No. of crypto combinations: 1231


,base,intermediate,ticker
0,USDT,BTC,ETH
1,USDT,BTC,LTC
2,USDT,BTC,BNB
3,USDT,BTC,NEO
4,USDT,BTC,BCC


# STEP 2: PERFORM TRIANGULAR ARBITRAGE

## Utility method to fetch the current ticker price

In [11]:
def fetch_current_ticker_price(tickerList):
    bidAsk = exchange.fetch_bids_asks(symbols= tickerList)

    # add 1 to limit counters
    requestPer10sec['requests'] += 1
    requestPerMin['requests'] += 1
    requestPer24h['requests'] += 1

    #if operation == 'buy':
    #    return (bidAsk[ticker]['ask'], bidAsk[ticker]['askVolume'])
    #else:
    #    return (bidAsk[ticker]['bid'], bidAsk[ticker]['bidVolume'])
    return bidAsk

## Triangular Arbitrage

In [12]:
def check_buy_buy_sell(scrip1, scrip2, scrip3,initial_investment):
    
    fullfetch = fetch_current_ticker_price([scrip1, scrip2, scrip3])
    if verbose == True:
        print(f'{scrip1}: {fullfetch[scrip1]["bid"]}/{fullfetch[scrip1]["bidVolume"]} - {fullfetch[scrip1]["ask"]}/{fullfetch[scrip1]["askVolume"]}')
        print(f'{scrip2}: {fullfetch[scrip2]["bid"]}/{fullfetch[scrip2]["bidVolume"]} - {fullfetch[scrip2]["ask"]}/{fullfetch[scrip2]["askVolume"]}')
        print(f'{scrip3}: {fullfetch[scrip3]["bid"]}/{fullfetch[scrip3]["bidVolume"]} - {fullfetch[scrip3]["ask"]}/{fullfetch[scrip3]["askVolume"]}')

    # add vol analysis for OP size from bidVolume & askVolume

    ## SCRIP1
    investment_amount1 = initial_investment
    current_price1 = fullfetch[scrip1]['ask'] #fetch_current_ticker_price(scrip1, 'buy')

    final_price = 0
    scrip_prices = {}
    #ZeroErr = False

    if (not current_price1 == None) or (not current_price1 == 0.0)\
        or (not current_price2 == None) or (not current_price2 == 0.0)\
        or (not current_price3 == None) or (not current_price3 == 0.0):
        try:
            buy_quantity1 = round(investment_amount1 / current_price1 * (1+marketFeesDF.loc[scrip1]['taker']), 8) #include commission in price
            #print(f'buy_quantity1 = {buy_quantity1}')
        except ZeroDivisionError:
            #print('Zero Division Err catch: buy_qty1 = 0')
            #ZeroErr = True
            buy_quantity1 = 0
        time.sleep(0.25)
        ## SCRIP2
        investment_amount2 = buy_quantity1
        current_price2 = fullfetch[scrip2]['ask'] #fetch_current_ticker_price(scrip2, 'buy')
    #if current_price2 is not None:
        try:
            buy_quantity2 = round(investment_amount2 / current_price2 * (1+marketFeesDF.loc[scrip2]['taker']), 8) #include commission in price
            #print(f'buy_quantity2 = {buy_quantity2}')
        except ZeroDivisionError:
            #print('Zero Division Err catch: buy_qty2 = 0')
            #ZeroErr = True
            buy_quantity2 = 0
        time.sleep(0.25)
        ## SCRIP3
        investment_amount3 = buy_quantity2
        current_price3 = fullfetch[scrip3]['bid'] #fetch_current_ticker_price(scrip3, 'sell')
    #if current_price3 is not None:
        sell_quantity3 = buy_quantity2
        final_price = round(sell_quantity3 * current_price3 * (1-marketFeesDF.loc[scrip3]['taker']),8) #include commission in price
        scrip_prices = {scrip1 : current_price1, scrip2 : current_price2, scrip3 : current_price3}
    #else:
    #    print('price3 is None')
    #else:
    #    print('price2 is None')
    else:
        print('some price is None or Zero')
        return None
    
    #if ZeroErr:
    #    print('Zero Err. catched')

    return final_price, scrip_prices

In [13]:
def check_buy_sell_sell(scrip1, scrip2, scrip3,initial_investment):
        
    fullfetch = fetch_current_ticker_price([scrip1, scrip2, scrip3])
    if verbose == True:
        print(f'{scrip1}: {fullfetch[scrip1]["bid"]}/{fullfetch[scrip1]["bidVolume"]} - {fullfetch[scrip1]["ask"]}/{fullfetch[scrip1]["askVolume"]}')
        print(f'{scrip2}: {fullfetch[scrip2]["bid"]}/{fullfetch[scrip2]["bidVolume"]} - {fullfetch[scrip2]["ask"]}/{fullfetch[scrip2]["askVolume"]}')
        print(f'{scrip3}: {fullfetch[scrip3]["bid"]}/{fullfetch[scrip3]["bidVolume"]} - {fullfetch[scrip3]["ask"]}/{fullfetch[scrip3]["askVolume"]}')

    # add vol analysis for OP size from bidVolume & askVolume

    ## SCRIP1
    investment_amount1 = initial_investment
    current_price1 = fullfetch[scrip1]['ask'] #fetch_current_ticker_price(scrip1, ' buy')
    
    final_price = 0
    scrip_prices = {}
    #ZeroErr = False
    if (not current_price1 == None) or (not current_price1 == 0)\
        or (not current_price2 == None) or (not current_price2 == 0)\
        or (not current_price3 == None) or (not current_price3 == 0):

        try:
            buy_quantity1 = round(investment_amount1 / current_price1 * (1+marketFeesDF.loc[scrip1]['taker']), 8) #include commission in price
            #print(f'buy_quantity1 = {buy_quantity1}')
        except ZeroDivisionError:
            #print('Zero Division Err catch: buy_qty1 = 0')
            #ZeroErr = True
            buy_quantity1 = 0

        time.sleep(0.25)
        ## SCRIP2
        investment_amount2 = buy_quantity1     
        current_price2 = fullfetch[scrip2]['bid'] #fetch_current_ticker_price(scrip2, 'sell')
    #if current_price2 is not None:
        sell_quantity2 = buy_quantity1
        sell_price2 = round(sell_quantity2 * current_price2 * (1-marketFeesDF.loc[scrip2]['taker']), 8) #include commission in price
        time.sleep(0.25)

        ## SCRIP3
        investment_amount3 = sell_price2     
        current_price3 = fullfetch[scrip3]['bid'] #fetch_current_ticker_price(scrip3, 'sell')
    #if current_price3 is not None:
        sell_quantity3 = sell_price2
        final_price = round(sell_quantity3 * current_price3 * (1-marketFeesDF.loc[scrip3]['taker']), 8) #include commission in price
        scrip_prices = {scrip1 : current_price1, scrip2 : current_price2, scrip3 : current_price3}
    #else:
    #    print('price3 is None')
    #else:
    #    print('price2 is None')
    else:
        print('some price is None or Zero')
        return None
    
    #if ZeroErr:
    #    print('Zero Err. catched')

    return final_price,scrip_prices


In [14]:
def check_profit_loss(total_price_after_sell,initial_investment, min_profit):
    min_profitable_price = initial_investment + min_profit
    profit_loss = round(total_price_after_sell - min_profitable_price,3)
    return profit_loss

# STEP 3: PLACE THE TRADE ORDERS

In [15]:
def place_buy_order(scrip, quantity, limit):
    order = exchange.create_limit_buy_order(scrip, quantity, limit)
    return order

def place_sell_order(scrip, quantity, limit):
    order = exchange.create_limit_sell_order(scrip, quantity, limit)
    return order 

def place_trade_orders(type, scrip1, scrip2, scrip3, initial_amount, scrip_prices):
    final_amount = 0.0
    if type == 'BUY_BUY_SELL':
        s1_quantity = initial_amount/scrip_prices[scrip1]
        place_buy_order(scrip1, s1_quantity, scrip_prices[scrip1])
        
        s2_quantity = s1_quantity/scrip_prices[scrip2]
        place_buy_order(scrip2, s2_quantity, scrip_prices[scrip2])
        
        s3_quantity = s2_quantity
        place_sell_order(scrip3, s3_quantity, scrip_prices[scrip3])
        
    elif type == 'BUY_SELL_SELL':
        s1_quantity = initial_amount/scrip_prices[scrip1]
        place_buy_order(scrip1, s1_quantity, scrip_prices[scrip1])
        
        s2_quantity = s1_quantity
        place_sell_order(scrip2, s2_quantity, scrip_prices[scrip2])
        
        s3_quantity = s2_quantity * scrip_prices[scrip2]
        place_sell_order(scrip3, s3_quantity, scrip_prices[scrip3])
        
        
    return final_amount

Sample order from exchange immediately after execution:   
{'info': {'id': '2490462375', 'symbol': 'btcusdt', 'type': 'limit', 'side': 'buy', 'status': 'wait', 'price': '43201.0', 'origQty': '0.002314', 'executedQty': '0.0', 'createdTime': '1646302254000', 'updatedTime': '1646302254000'}, 'id': '2490462375', 'clientOrderId': None, 'timestamp': 1646302254000, 'datetime': '2022-03-03T10:10:54.000Z', 'lastTradeTimestamp': 1646302254000, 'status': 'open', 'symbol': 'BTC/USDT', 'type': 'limit', 'timeInForce': None, 'postOnly': None, 'side': 'buy', 'price': 43201.0, 'amount': None, 'filled': 0.0, 'remaining': None, 'cost': 0.0, 'fee': None, 'average': None, 'trades': [], 'fees': []}

# STEP 4: WRAPPING IT TOGETHER

In [16]:
#change datetime.now() for timestamp

def perform_triangular_arbitrage(scrip1, scrip2, scrip3, arbitrage_type,initial_investment, min_profit):

    final_price = 0.0
    if(arbitrage_type == 'BUY_BUY_SELL'):
        # Check this combination for triangular arbitrage: scrip1 - BUY, scrip2 - BUY, scrip3 - SELL
        final_price, scrip_prices = check_buy_buy_sell(scrip1, scrip2, scrip3,initial_investment)
        
    elif(arbitrage_type == 'BUY_SELL_SELL'):
        # Check this combination for triangular arbitrage: scrip1 - BUY, scrip2 - SELL, scrip3 - SELL
        final_price, scrip_prices = check_buy_sell_sell(scrip1, scrip2, scrip3,initial_investment)
        
    profit_loss = check_profit_loss(final_price,initial_investment, min_profit)
    result = f"{dt.datetime.now().strftime('%d-%b-%Y %H:%M:%S.%f')},"\
              f"{arbitrage_type}, {scrip1}, {scrip_prices[scrip1]}, {scrip2}, {scrip_prices[scrip2]}, {scrip3}, {scrip_prices[scrip3]}, {round(final_price-initial_investment,3)}"

    if profit_loss>0:
        #place_trade_orders(arbitrage_type, scrip1, scrip2, scrip3, initial_investment, scrip_prices)
        print(result)
    return result

### Set request strict limits

In [17]:
#Límites estrictos:
#Ponderación de 1200 solicitudes por minuto (ten en cuenta que no es necesariamente lo mismo que 1200 solicitudes)
#50 órdenes cada 10 segundos
#160 000 órdenes cada 24 horas
#Nuestros límites estrictos están incluidos en el punto final [/api/v3/exchangeInfo].
placeholderTime = dt.datetime.strptime('00:00', '%H:%M')
requestPerMin = {'start': placeholderTime , 'end': placeholderTime, 'requests': 0}
requestPer10sec = {'start': placeholderTime, 'end': placeholderTime, 'requests': 0}
requestPer24h = {'start': placeholderTime, 'end': placeholderTime, 'requests': 0}

limit10sec = False
limitPerMin = False
limitPer24H = False

# Calculate average request per minute
def AVGrequestsPerMin(requests, start, end = dt.datetime.now()):
    interval = end - start
    avg = round(requests/ round(interval.total_seconds()/60,0), 0)
    return avg

In [18]:
def checkLimits(verbose= False):
    if requestPer24h['requests'] > (160000-3):
        deltaToEnd = requestPer24h['end'] - dt.datetime.now()
        print(f'24H limit reached, spleeping {deltaToEnd} until {requestPer24h["end"]}')
        time.sleep(deltaToEnd.total_seconds())
    elif requestPer24h['end'] < dt.datetime.now():
        requestPer24h['start'] = dt.datetime.now()
        requestPer24h['end'] = requestPer24h['start'] + dt.timedelta(hours= 24)
        requestPer24h['requests'] = 0
        if verbose == True:
            print('24H counter reset')

    if requestPer10sec['requests'] > (50-3):
        deltaToEnd = requestPer10sec['end'] - dt.datetime.now()
        print(f'10 sec. limit reached, sleeping {deltaToEnd} until {requestPer10sec["end"]}')
        time.sleep(deltaToEnd.total_seconds())
    elif requestPer10sec['end'] < dt.datetime.now():
        requestPer10sec['start'] = dt.datetime.now()
        requestPer10sec['end'] = requestPer10sec['start'] + dt.timedelta(seconds= 10)
        requestPer10sec['requests'] = 0
        if verbose == True:
            print('10 sec counter reset')
    
    avgMin = AVGrequestsPerMin(requestPerMin['requests'],requestPerMin['start'])
    if avgMin > 1200:
        print(f'Request per minute limit reached, sleeping 1 minute')
        time.sleep(60)
    
    if verbose == True:
        # print limit counters
        print(f'Limit 10 sec: {requestPer10sec["requests"]}')
        print(f'Limit min AVG: {avgMin}')
        print(f'Limit 24h: {requestPer24h["requests"]}')
    

In [21]:
# brokerage commission from API included in price @ check bbs or sbb strategies
verbose = False
INVESTMENT_AMOUNT_DOLLARS = 100
MIN_PROFIT_DOLLARS = 0.5
#BROKERAGE_PER_TRANSACTION_PERCENT = 0.2 ## taken from marketFeesDF

#while(1):
for combination in wx_combinations_usdt:

    base = combination['base']
    intermediate = combination['intermediate']
    ticker = combination['ticker']


    s1 = f'{intermediate}/{base}'    # Eg: BTC/USDT
    s2 = f'{ticker}/{intermediate}'  # Eg: ETH/BTC
    s3 = f'{ticker}/{base}'          # Eg: ETH/USDT
    

    
    # check request limits
    checkLimits(verbose= True)

    # Check triangular arbitrage for buy-buy-sell 
    bbs = perform_triangular_arbitrage(s1,s2,s3,'BUY_BUY_SELL',INVESTMENT_AMOUNT_DOLLARS, MIN_PROFIT_DOLLARS)
    # Sleep to avoid rate limit on api calls (RateLimitExceeded exception)
    if not bbs == None:
        with open(f'TriBot_output_{dt.datetime.today().date().strftime("%d%m%Y")}.csv', 'a') as f:
            f.write(bbs+'\n')

    # check request limits
    checkLimits(verbose= True)

    # Check triangular arbitrage for buy-sell-sell 
    bss = perform_triangular_arbitrage(s3,s2,s1,'BUY_SELL_SELL',INVESTMENT_AMOUNT_DOLLARS, MIN_PROFIT_DOLLARS)
    if not bss == None:
        with open(f'TriBot_output_{dt.datetime.today().date().strftime("%d%m%Y")}.csv', 'a') as f:
            f.write(bss+'\n')


10 sec counter reset
Limit 10 sec: 0
Limit min AVG: 0.0
Limit 24h: 798
Limit 10 sec: 1
Limit min AVG: 0.0
Limit 24h: 799
Limit 10 sec: 2
Limit min AVG: 0.0
Limit 24h: 800
Limit 10 sec: 3
Limit min AVG: 0.0
Limit 24h: 801
Limit 10 sec: 4
Limit min AVG: 0.0
Limit 24h: 802
Limit 10 sec: 5
Limit min AVG: 0.0
Limit 24h: 803
Limit 10 sec: 6
Limit min AVG: 0.0
Limit 24h: 804
Limit 10 sec: 7
Limit min AVG: 0.0
Limit 24h: 805
Limit 10 sec: 8
Limit min AVG: 0.0
Limit 24h: 806


C:\Users\jbarrero\AppData\Local\Temp\1\ipykernel_24972\3418482996.py:35: RuntimeWarning: divide by zero encountered in double_scalars
  buy_quantity2 = round(investment_amount2 / current_price2 * (1+marketFeesDF.loc[scrip2]['taker']), 8) #include commission in price
C:\Users\jbarrero\AppData\Local\Temp\1\ipykernel_24972\3418482996.py:47: RuntimeWarning: invalid value encountered in double_scalars
  final_price = round(sell_quantity3 * current_price3 * (1-marketFeesDF.loc[scrip3]['taker']),8) #include commission in price


Limit 10 sec: 9
Limit min AVG: 0.0
Limit 24h: 807
Limit 10 sec: 10
Limit min AVG: 0.0
Limit 24h: 808
Limit 10 sec: 11
Limit min AVG: 0.0
Limit 24h: 809
10 sec counter reset
Limit 10 sec: 0
Limit min AVG: 0.0
Limit 24h: 810
Limit 10 sec: 1
Limit min AVG: 0.0
Limit 24h: 811
Limit 10 sec: 2
Limit min AVG: 0.0
Limit 24h: 812
Limit 10 sec: 3
Limit min AVG: 0.0
Limit 24h: 813
Limit 10 sec: 4
Limit min AVG: 0.0
Limit 24h: 814
Limit 10 sec: 5
Limit min AVG: 0.0
Limit 24h: 815
Limit 10 sec: 6
Limit min AVG: 0.0
Limit 24h: 816
Limit 10 sec: 7
Limit min AVG: 0.0
Limit 24h: 817
Limit 10 sec: 8
Limit min AVG: 0.0
Limit 24h: 818
Limit 10 sec: 9
Limit min AVG: 0.0
Limit 24h: 819
10 sec counter reset
Limit 10 sec: 0
Limit min AVG: 0.0
Limit 24h: 820
Limit 10 sec: 1
Limit min AVG: 0.0
Limit 24h: 821
Limit 10 sec: 2
Limit min AVG: 0.0
Limit 24h: 822
Limit 10 sec: 3
Limit min AVG: 0.0
Limit 24h: 823
Limit 10 sec: 4
Limit min AVG: 0.0
Limit 24h: 824
03-May-2022 00:53:02.273929,BUY_BUY_SELL, BTC/USDT, 3862